In [1]:
from pycalphad import Database, Model
from pycalphad.core.utils import make_callable
import pycalphad.variables as v
import numba
import numpy as np

In [2]:
dbf = Database('Al-Ni/Al-Ni-Dupin-2001.tdb')
mod = Model(dbf, ['AL', 'NI', 'VA'], 'FCC_L12')

In [3]:
from sympy import lambdify
from pycalphad.core.utils import NumPyPrinter
@numba.vectorize
def where(condition, x, y):
    if condition:
        return x
    else:
        return y
func = lambdify(tuple(mod.variables), mod.ast, dummify=True,
                          modules=[{'where': where}, 'numpy'], printer=NumPyPrinter)
pure_func = lambdify(tuple(mod.variables), mod.ast, dummify=True,
                          modules=['numpy'], printer=NumPyPrinter)

In [4]:
vec_func = numba.vectorize(['float64({})'.format(','.join(['float64'] * len(tuple(mod.variables))))], nopython=True, target='parallel')(func)


x1 = np.random.rand(5, 1, 1)*2000
x2 = np.random.rand(5, 5, 3000)
x3 = np.random.rand(5, 5, 3000)
x4 = np.random.rand(5, 5, 3000)
x5 = np.random.rand(5, 5, 3000)
x6 = np.random.rand(5, 5, 3000)
#np.testing.assert_allclose(vec_func(x1, x2, x3, x4, x5, x6), pure_func(x1, x2, x3, x4, x5, x6))
#%timeit vec_func(x1, x2, x3, x4, x5, x6)

In [22]:
from sympy import lambdify
import numba
def make_gradient_from_graph(graph, wrt):
    wrt = tuple(wrt)
    grads = np.empty((len(wrt)), dtype=object)
    hess_indices = []
    namespace = {}
    for i in range(len(wrt)):
        grads[i] = graph.diff(wrt[i])
        for j in range(i, len(wrt)):
            namespace['hess_{0}{1}'.format(i, j)] = numba.vectorize(lambdify(tuple(wrt), grads[i].diff(wrt[j]), dummify=True,
                                                              modules=[{'where': where}, 'numpy'], printer=NumPyPrinter))
            hess_indices.append((i, j))
        namespace['grad_{0}'.format(i)] = numba.vectorize(lambdify(tuple(wrt), grads[i], dummify=True,
                                                    modules=[{'where': where}, 'numpy'], printer=NumPyPrinter))
    # Build the gradient and Hessian using compile() and exec
    # We do this because Numba needs "static" information about the arguments and functions
    call_args = ','.join(['_x{0}'.format(i) for i in range(len(wrt))])
    call_passed_args = ','.join(['_x{0}[0]'.format(i) for i in range(len(wrt))])

    grad_code = 'def grad_func({0}, lengthfix, result):'.format(call_args)
    grad_list = ['    result[{0}] = grad_{0}({1})'.format(i, call_passed_args) for i in range(len(wrt))]
    grad_code = grad_code + '\n' + '\n'.join(grad_list)
    print(grad_code)
    
    grad_code = compile(grad_code, '<string>', 'exec')
    exec grad_code in namespace

    # Now construct the Hessian
    hess_code = 'def hess_func({0}, lengthfix, result):'.format(call_args)
    hess_list = ['    result[{0},{1}] = result[{1}, {0}] = hess_{0}{1}({2})'.format(i, j, call_passed_args) for i, j in hess_indices]
    hess_code = hess_code + '\n' + '\n'.join(hess_list)
    print(hess_code)
    
    hess_code = compile(hess_code, '<string>', 'exec')
    exec hess_code in namespace

    grad_func = numba.guvectorize([','.join(['float64[:]'] * (len(wrt)+2))],
                                   ','.join(['()'] * len(wrt)) + ',(n)->(n)', nopython=True)(namespace['grad_func'])
    hess_func = numba.guvectorize([','.join(['float64[:]'] * (len(wrt)+1)) + ',float64[:,:]'],
                                   ','.join(['()'] * len(wrt)) + ',(n)->(n,n)', nopython=True)(namespace['hess_func'])
    return grad_func, hess_func

In [23]:
%time grad_func, hess_func = make_gradient_from_graph(mod.ast, mod.variables)

def grad_func(_x0,_x1,_x2,_x3,_x4,_x5, lengthfix, result):
    result[0] = grad_0(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[1] = grad_1(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[2] = grad_2(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[3] = grad_3(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[4] = grad_4(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[5] = grad_5(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
def hess_func(_x0,_x1,_x2,_x3,_x4,_x5, lengthfix, result):
    result[0,0] = result[0, 0] = hess_00(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[0,1] = result[1, 0] = hess_01(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[0,2] = result[2, 0] = hess_02(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[0,3] = result[3, 0] = hess_03(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[0,4] = result[4, 0] = hess_04(_x0[0],_x1[0],_x2[0],_x3[0],_x4[0],_x5[0])
    result[0,5] = result[5, 0] = hess_05(_x0[0],_x1[0],_x2[0],_x3[

In [24]:
%time grad_func(x1, x2, x3, x4, x5, x6, np.empty(6), np.empty(x6.shape + (6,)))

CPU times: user 130 ms, sys: 0 ns, total: 130 ms
Wall time: 129 ms


array([[[[ -2.33212125e+01,  -5.36832606e+03,  -1.49158148e+04,
           -2.11298180e+04,  -2.23344599e+04,  -5.67704297e+04],
         [ -3.75829968e+01,  -8.32937199e+03,  -1.87695977e+04,
           -3.98934407e+04,  -4.11798979e+04,  -1.06274990e+05],
         [ -4.56736213e+01,  -1.06891890e+04,  -3.55824578e+04,
           -3.73301291e+04,  -4.67603182e+04,  -1.00191370e+05],
         ..., 
         [ -2.16156897e+01,  -2.03573573e+04,  -8.42319125e+03,
           -2.66323526e+04,  -1.95150855e+04,  -7.35335140e+04],
         [ -1.56368680e+01,  -1.59148513e+03,  -1.26038410e+03,
           -1.73469405e+04,  -1.51941367e+04,  -3.48687251e+04],
         [ -2.28003748e+01,  -1.17088751e+04,  -3.53854494e+04,
           -1.15731699e+04,  -1.70976540e+04,  -4.82762518e+04]],

        [[ -3.42651759e+01,  -1.12853063e+04,  -2.29436171e+04,
           -3.18517067e+04,  -3.47357442e+04,  -9.15827379e+04],
         [ -2.99331956e+00,  -6.97252803e+03,   5.44645838e+02,
           -9.40

In [26]:
result = np.zeros(x6.shape + (6,6))
%time hess_func(x1, x2, x3, x4, x5, x6, np.empty(6), result)
print(result)

CPU times: user 342 ms, sys: 7 ms, total: 349 ms
Wall time: 347 ms
[[[[[ -2.03298325e-02  -1.42391923e+01  -8.78351681e+00  -8.14509029e-01
      -1.23907921e+00  -5.45246223e+01]
    [ -1.42391923e+01   3.68504602e+04  -2.60720978e+02  -9.48984017e+03
      -2.66966492e+04  -1.65588736e+04]
    [ -8.78351681e+00  -2.60720978e+02   1.85383980e+04  -2.83455971e+04
      -1.43115770e+04  -4.20604230e+04]
    [ -8.14509029e-01  -9.48984017e+03  -2.83455971e+04   6.05513981e+03
       3.16021807e+03  -5.49553873e+04]
    [ -1.23907921e+00  -2.66966492e+04  -1.43115770e+04   3.16021807e+03
       3.92236856e+03  -5.69438261e+04]
    [ -5.45246223e+01  -1.65588736e+04  -4.20604230e+04  -5.49553873e+04
      -5.69438261e+04  -7.68856036e+04]]

   [[ -3.64667492e-02  -2.17052236e+01  -1.32267888e+01  -1.44862750e+00
       2.53322706e+00  -8.31413249e+01]
    [ -2.17052236e+01   6.30456110e+04   1.72689603e+03  -9.95273206e+03
      -3.21399741e+04  -2.29344856e+04]
    [ -1.32267888e+01   1.7

In [ ]:
#%timeit pure_func(x1, x2, x3, x4, x5, x6)

In [ ]:
print(numba.typeof(grads))